In [ ]:
import os
import time
import random
import sqlite3
import requests

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
tqdm.pandas()

from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent

os.environ["OPENAI_API_KEY"] = "your key here"

In [2]:
process_adaptations_dict = { 
    "base_rule": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_base_rule.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_base_rule.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_base_rule.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_base_rule.bpmn and store it in the database as J09D",
    },
    "0_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_0_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_0_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_0_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_0_values.bpmn and store it in the database as J09D",
    },
    "500_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_500_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_500_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_500_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_500_values.bpmn and store it in the database as J09D",
    },
    "900_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_900_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_900_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_900_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_900_values.bpmn and store it in the database as J09D",
    },
    "city_values": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_city_values.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_city_values.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_city_values.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_city_values.bpmn and store it in the database as J09D",
    },
    "extension_estimates": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_extension_estimates.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_extension_estimates.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_extension_estimates.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_extension_estimates.bpmn and store it in the database as J09D",
    },
    "extension_mail": {
        "J09A": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09A_extension_mail.bpmn and store it in the database as J09A",
        "J09B": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09B_extension_mail.bpmn and store it in the database as J09B",
        "J09C": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09C_extension_mail.bpmn and store it in the database as J09C",
        "J09D": "Use the generate_process_rule_from_bpmn tool to generate a process rule from the following BPMN file: bpmn/J09D_extension_mail.bpmn and store it in the database as J09D",
    },
}

In [3]:
random.seed(42)
SEEDS = [42,]
SEEDS += random.sample(range(0, 10000), 4)
print(SEEDS)

[42, 1824, 409, 4506, 4012]


In [4]:
from itertools import product
process_adaptations = process_adaptations_dict.keys()
rule_adaptation_methods = ["generate_bpmn", ] # "generate_bpmn"
tours = ["J09A", "J09B", "J09C", "J09D"]
runs = list(product(process_adaptations, rule_adaptation_methods, tours, SEEDS))
runs

[('base_rule', 'generate_bpmn', 'J09A', 42),
 ('base_rule', 'generate_bpmn', 'J09A', 1824),
 ('base_rule', 'generate_bpmn', 'J09A', 409),
 ('base_rule', 'generate_bpmn', 'J09A', 4506),
 ('base_rule', 'generate_bpmn', 'J09A', 4012),
 ('base_rule', 'generate_bpmn', 'J09B', 42),
 ('base_rule', 'generate_bpmn', 'J09B', 1824),
 ('base_rule', 'generate_bpmn', 'J09B', 409),
 ('base_rule', 'generate_bpmn', 'J09B', 4506),
 ('base_rule', 'generate_bpmn', 'J09B', 4012),
 ('base_rule', 'generate_bpmn', 'J09C', 42),
 ('base_rule', 'generate_bpmn', 'J09C', 1824),
 ('base_rule', 'generate_bpmn', 'J09C', 409),
 ('base_rule', 'generate_bpmn', 'J09C', 4506),
 ('base_rule', 'generate_bpmn', 'J09C', 4012),
 ('base_rule', 'generate_bpmn', 'J09D', 42),
 ('base_rule', 'generate_bpmn', 'J09D', 1824),
 ('base_rule', 'generate_bpmn', 'J09D', 409),
 ('base_rule', 'generate_bpmn', 'J09D', 4506),
 ('base_rule', 'generate_bpmn', 'J09D', 4012),
 ('0_values', 'generate_bpmn', 'J09A', 42),
 ('0_values', 'generate_bpmn

In [5]:
for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}"
    if os.path.exists(f"sessions/{session_id}/"):
        print(f"Session {session_id} already exists. Skipping...")
        continue

    with open("seed.txt", "w", encoding="utf-8") as f:
        f.write(str(seed))
    
    BASE_FRAME_TEMPLATE = process_adaptations_dict.get(process_adaptation).get(tour)
    
    # Drop process_rules table to start fresh each time
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS process_rules")
    conn.commit()
    conn.close()
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE process_rules (
            process_rule_id TEXT PRIMARY KEY,
            process_rule TEXT NOT NULL
        )
    """)
    conn.commit()
    conn.close()
    
    client = MultiServerMCPClient(  
        {
            "fastapi": {
                "transport": "streamable_http",  # HTTP-based remote server
                # Ensure you start your fastapi server on port 8000
                "url": "http://localhost:8001/mcp",
            }
        }
    )
    tools = await client.get_tools()

    SYSTEM_PROMPT = """You are a process-frame agent with three main capabilities. 
First, you can take a BPMN file path and generate a structured process rule from it, automatically storing it in the rule database along with an ID. 
Second, you can take a natural language description from a human and also convert that into a structured process rule, automatically writing it to the database with an ID. 
Third, if a user requests that a specific process ID be run, you can call a process-execution agent to handle that execution. 
Always follow these steps and ensure each rule is stored and each process is run as requested."""

    config = {
        "model": "gpt-5.1",
        #"service_tier": "priority",
        "temperature": 0,
        "seed": seed,
        "reasoning_effort": "high",
    }

    model = ChatOpenAI(
        **config
    )

    frame_agent = create_agent(
        model,
        tools=tools,
        system_prompt=SYSTEM_PROMPT,
    )
    
    # Add process rule to the database
    print(f"Adding process rule for session {session_id}...")
    rule_add_prompt = ""

    if rule_adaptation_method == "add":
        PROMPT_TEMPLATE = """**Add** this process rule with id '{tour}' to the rule database:

{BASE_FRAME_TEMPLATE_FILLED}"""
        prompt = PROMPT_TEMPLATE.format(tour=tour, BASE_FRAME_TEMPLATE_FILLED=BASE_FRAME_TEMPLATE.format(tour=tour))
    elif rule_adaptation_method == "generate_bpmn":
        prompt = BASE_FRAME_TEMPLATE
    with open(f"session_id.txt", "w", encoding="utf-8") as f:
        f.write(session_id)

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_addition.txt", "w", encoding="utf-8") as f:    
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

    
    # After adding the rule, we can verify it's in the database
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM process_rules WHERE process_rule_id = ?", (tour,))
    row = cursor.fetchone()
    if row is not None:
        rule_added = True
    else:
        rule_added = False
    conn.close()

    with open(f"sessions/{session_id}/frame_metadata.csv", "w", encoding="utf-8") as f:
        f.write(f"total_time_seconds,rule_added\n")
        f.write(f"{et},{rule_added}\n")
    
    # Run process rule

    print(f"Running process rule for session {session_id}...")
    
    process_prompt = f"""Run process_rule with process_rule_id '{tour}'."""

    st = time.time()

    response = await frame_agent.ainvoke(
        {"messages": [{"role": "user", "content": process_prompt}]}
    )

    os.makedirs(f"sessions/{session_id}", exist_ok=True)
    with open(f"sessions/{session_id}/frame_agent_process_execution.txt", "w", encoding="utf-8") as f:
        for m in response["messages"]:
            f.write(m.pretty_repr() + "\n")
    
    et = time.time() - st
    print(f"Total time: {et} seconds")

Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_42 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_1824 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_409 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_4506 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_4012 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09B_seed_42 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09B_seed_1824 already exists. Skipping...
Session processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09B_seed_409 already exists. Skipping...
Session processadaptation_base

In [6]:
process_results_columns = ["session_id", "process_adaptation", "rule_adaptation_method", "tour", "seed", "correct_matches", "correct_columns", "file_extracted", "mails_sent", "all_correct"]
process_results = []

for run in runs:
    process_adaptation, rule_adaptation_method, tour, seed = run
    
    session_id = f"processadaptation_{process_adaptation}_processadaptationmethod_{rule_adaptation_method}_tour_{tour}_seed_{seed}"
    print(f"Session ID: {session_id}")

    df_original = pd.read_csv(f"data/XML {tour}.csv").fillna("")
    
    # Load original decision table    
    if process_adaptation == "base_rule":
        df_decision = pd.read_csv(f"data/XML {tour}.csv").fillna("")
        equipment_count_per_house = df_decision.groupby('HAUS')['EQUNR'].nunique().reset_index()
        equipment_count_per_house.columns = ['HAUS', 'equipment_count']
        df_original = df_original.merge(equipment_count_per_house, on='HAUS', how='inner')
        df_original["ENTSCHEIDUNG"] = df_original["equipment_count"].apply(lambda x: "KSA" if x < 3 else "EVU")
    elif process_adaptation == "0_values":
        df_decision = pd.read_csv(f"data/XML {tour}.csv").fillna("")
        equipment_count_per_house = df_decision.groupby('HAUS')['EQUNR'].nunique().reset_index()
        equipment_count_per_house.columns = ['HAUS', 'equipment_count']
        df_original = df_original.merge(equipment_count_per_house, on='HAUS', how='inner')
        df_original["ENTSCHEIDUNG"] = df_original["equipment_count"].apply(lambda x: "KSA" if x < 3 else "EVU")
        df_original["HOUSE_NUM1"] = df_original["HOUSE_NUM1"].apply(lambda x: "1111" if x == "" else x) # for empty HOUSE_NUM1 values to prevent false zeros
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "EVU" if int(row["HOUSE_NUM1"]) == 0 else row["ENTSCHEIDUNG"], axis=1)
    elif process_adaptation == "500_values":
        df_decision = pd.read_csv(f"data/XML {tour}.csv").fillna("")
        equipment_count_per_house = df_decision.groupby('HAUS')['EQUNR'].nunique().reset_index()
        equipment_count_per_house.columns = ['HAUS', 'equipment_count']
        df_original = df_original.merge(equipment_count_per_house, on='HAUS', how='inner')
        df_original["ENTSCHEIDUNG"] = df_original["equipment_count"].apply(lambda x: "KSA" if x < 3 else "EVU")
        df_original["HOUSE_NUM1"] = df_original["HOUSE_NUM1"].apply(lambda x: "1111" if x == "" else x) # for empty HOUSE_NUM1 values to prevent false zeros
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "EVU" if int(row["HOUSE_NUM1"]) == 0 or (500 <= int(row["HOUSE_NUM1"]) <= 599) else row["ENTSCHEIDUNG"], axis=1)
    elif process_adaptation == "900_values":
        df_decision = pd.read_csv(f"data/XML {tour}.csv").fillna("")
        equipment_count_per_house = df_decision.groupby('HAUS')['EQUNR'].nunique().reset_index()
        equipment_count_per_house.columns = ['HAUS', 'equipment_count']
        df_original = df_original.merge(equipment_count_per_house, on='HAUS', how='inner')
        df_original["ENTSCHEIDUNG"] = df_original["equipment_count"].apply(lambda x: "KSA" if x < 3 else "EVU")
        df_original["HOUSE_NUM1"] = df_original["HOUSE_NUM1"].apply(lambda x: "1111" if x == "" else x) # for empty HOUSE_NUM1 values to prevent false zeros
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "EVU" if int(row["HOUSE_NUM1"]) == 0 or (500 <= int(row["HOUSE_NUM1"]) <= 599) or (900 <= int(row["HOUSE_NUM1"]) <= 999) else row["ENTSCHEIDUNG"], axis=1)
    elif process_adaptation == "extension_estimates":
        df_eabl = pd.read_csv(f"data/EABL {tour}.csv").fillna("")
        df_eabl_2023 = df_eabl[(((df_eabl["ISTABLART_TXT"] == "Maschinelle Schätzung - SAP") & (df_eabl["Record created on"].str.contains("2024"))))]
        df_eabl_2024 = df_eabl[(((df_eabl["ISTABLART_TXT"] == "Maschinelle Schätzung - SAP") & (df_eabl["Record created on"].str.contains("2023"))))]
        df_eabl_2023_2024 = df_eabl_2023.merge(df_eabl_2024, on="EQUNR", suffixes=("_2023", "_2024"), how="inner")
        mandatory_equnr = df_eabl_2023_2024['EQUNR'].unique().tolist()
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "MANDATORY_READING" if row["EQUNR"] in mandatory_equnr else row["ENTSCHEIDUNG"], axis=1)
    elif process_adaptation == "extension_mail":
        df_eabl = pd.read_csv(f"data/EABL {tour}.csv").fillna("")
        df_eabl_2023 = df_eabl[(((df_eabl["ISTABLART_TXT"] == "Maschinelle Schätzung - SAP") & (df_eabl["Record created on"].str.contains("2024"))))]
        df_eabl_2024 = df_eabl[(((df_eabl["ISTABLART_TXT"] == "Maschinelle Schätzung - SAP") & (df_eabl["Record created on"].str.contains("2023"))))]
        df_eabl_2023_2024 = df_eabl_2023.merge(df_eabl_2024, on="EQUNR", suffixes=("_2023", "_2024"), how="inner")
        mandatory_equnr = df_eabl_2023_2024['EQUNR'].unique().tolist()
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "MANDATORY_READING" if row["EQUNR"] in mandatory_equnr else row["ENTSCHEIDUNG"], axis=1)
        direct_mail_equnr_2023 = df_eabl[((df_eabl["ISTABLART_TXT"] == "Ablesung durch Kunden - SAP") & (df_eabl["Record created on"].str.contains("2024")))]
        direct_mail_equnr_2024 = df_eabl[((df_eabl["ISTABLART_TXT"] == "Ablesung durch Kunden - SAP") & (df_eabl["Record created on"].str.contains("2023")))]
        direct_mail_equnr = direct_mail_equnr_2023.merge(direct_mail_equnr_2024, on="EQUNR", suffixes=("_2023", "_2024"), how="inner")['EQUNR'].unique().tolist()
        df_original["ENTSCHEIDUNG"] = df_original.apply(lambda row: "DIRECT_MAIL" if row["EQUNR"] in direct_mail_equnr else row["ENTSCHEIDUNG"], axis=1)
        df_original = df_original.merge(df_eabl[['Meter Reading (MR) Doc. No.', 'EQUNR']], left_on="EQUNR", right_on="EQUNR", how='left')

    correct_matches = False
    correct_columns = False
    file_extracted = False
    mails_sent = False

    try:
        file_extracted = False
        # Load session output
        df_meta_session = pd.read_csv(f"sessions/{session_id}/process_metadata.csv")
        session_file_path = df_meta_session.loc[0, 'file_path']
        df_session = pd.read_csv(session_file_path).fillna("")
        if process_adaptation in ["base_rule", "0_values", "500_values", "900_values", "city_values"]:
            file_extracted = True # if the file couls be loaded, then it was extracted
        if process_adaptation == "extension_estimates":
            df_session1 = pd.read_csv(f"sessions/{session_id}/{tour}_MANDATORY_READING.csv").fillna("")
            df_session = pd.concat([df_session, df_session1], ignore_index=True)
            df_session = df_session.reset_index(drop=True)
            file_extracted = True # extra check for mandatory reading file
        if process_adaptation == "extension_mail":
            df_session1 = pd.read_csv(f"sessions/{session_id}/{tour}_MANDATORY_READING.csv").fillna("")
            df_session2 = pd.read_csv(f"sessions/{session_id}/{tour}_DIRECT_MAIL.csv").fillna("")
            df_session = pd.concat([df_session, df_session1, df_session2], ignore_index=True)
            df_session = df_session.reset_index(drop=True)
            file_extracted = True # extra check for mandatory reading and direct mail files
        
        join_key = 'Meter Reading (MR) Doc. No.' if 'Meter Reading (MR) Doc. No.' in df_original.columns else "ME_MA_ID" if 'ME_MA_ID' in df_original.columns else None
        
        # Merge the dataframes
        df_comparison = df_original.merge(
            df_session,
            on=join_key,
            how='inner',
            suffixes=('_original', '_session')
        )
        df_comparison["match"] = df_comparison.apply(lambda row: row["ENTSCHEIDUNG_original"] == row["ENTSCHEIDUNG_session"], axis=1)
        try:
            display(df_comparison[df_comparison["match"] == False][["Meter Reading (MR) Doc. No.", "HAUS_session", "EQUNR_session", "HOUSE_NUM1_session", "REGEL", "ENTSCHEIDUNG_session", "ENTSCHEIDUNG_original"]].copy())
        except Exception as e:
            print(f"Error displaying mismatches: {e}")
        correct_matches = df_comparison["match"].all()
        correct_columns = [True if x +"_original" in df_comparison.columns or x+"_session" in df_comparison.columns or x in df_comparison.columns else False for x in ["Meter Reading (MR) Doc. No.","HAUS","ANLAGE","ME_MA_ID","EQUNR","HOUSE_NUM1","ENTSCHEIDUNG","TITLE","FORENAME","SURNAME","STREET","STREETNO","POSTCODE1","CITY1","CITY2","EMAIL"]]
        correct_columns = all(correct_columns)
        try:
            df_mails = pd.read_csv(f"sessions/{session_id}/process_metadata.csv")
            if len(df_mails) > 0 and process_adaptation in ["base_rule", "0_values", "500_values", "900_values", "city_values"]:
                if f"{tour}.csv" in df_mails.loc[0, "file_path"] and df_mails.loc[0, "email_address"] == "meter.readings@evu.com":
                    mails_sent = True
            if len(df_mails) > 1 and process_adaptation == "extension_estimates":
                if f"{tour}_MANDATORY_READING.csv" in df_mails.loc[0, "file_path"] and df_mails.loc[0, "email_address"] == "mandatory.reading@provider.com" and f"{tour}.csv" in df_mails.loc[2, "file_path"] and df_mails.loc[2, "email_address"] == "meter.readings@evu.com" or f"{tour}_MANDATORY_READING.csv" in df_mails.loc[2, "file_path"] and df_mails.loc[2, "email_address"] == "mandatory.reading@provider.com" and f"{tour}.csv" in df_mails.loc[0, "file_path"] and df_mails.loc[0, "email_address"] == "meter.readings@evu.com":
                    mails_sent = True
            if len(df_mails) > 1 and process_adaptation == "extension_mail":
                if f"{tour}_MANDATORY_READING.csv" in df_mails.loc[0, "file_path"] and df_mails.loc[0, "email_address"] == "mandatory.reading@provider.com" and f"{tour}.csv" in df_mails.loc[2, "file_path"] and df_mails.loc[2, "email_address"] == "meter.readings@evu.com":
                    with open(f"sessions/{session_id}/process_agent.txt", "r", encoding="utf-8") as f:
                        process_agent_log = f.read()
                    if "Name: send_bulk_mail" in process_agent_log:
                        mails_sent = True
        except Exception as e:
            print(f"Error checking mails sent: {e}")
    except Exception as e:
        print(f"Error processing session {session_id}: {e}")
    all_correct = correct_matches and correct_columns and file_extracted and mails_sent
    process_results.append([
        session_id,
        process_adaptation,
        rule_adaptation_method,
        tour,
        seed,
        correct_matches,
        correct_columns,
        file_extracted,
        mails_sent,
        all_correct
    ])
df_results_add_rules = pd.DataFrame(process_results, columns=process_results_columns)
df_results_add_rules.to_csv("process_adaptation_results_generate_from_bpmn_rules.csv", index=False)
display(df_results_add_rules)
display(df_results_add_rules[df_results_add_rules["all_correct"] == False])

Session ID: processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_42


,Meter Reading (MR) Doc. No.,HAUS_session,EQUNR_session,HOUSE_NUM1_session,REGEL,ENTSCHEIDUNG_session,ENTSCHEIDUNG_original


Session ID: processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_1824


,Meter Reading (MR) Doc. No.,HAUS_session,EQUNR_session,HOUSE_NUM1_session,REGEL,ENTSCHEIDUNG_session,ENTSCHEIDUNG_original


Session ID: processadaptation_base_rule_processadaptationmethod_generate_bpmn_tour_J09A_seed_409


...

,Meter Reading (MR) Doc. No.,HAUS_session,EQUNR_session,HOUSE_NUM1_session,REGEL,ENTSCHEIDUNG_session,ENTSCHEIDUNG_original


Session ID: processadaptation_extension_mail_processadaptationmethod_generate_bpmn_tour_J09D_seed_4506
Error processing session processadaptation_extension_mail_processadaptationmethod_generate_bpmn_tour_J09D_seed_4506: [Errno 2] No such file or directory: 'sessions/processadaptation_extension_mail_processadaptationmethod_generate_bpmn_tour_J09D_seed_4506/process_metadata.csv'
Session ID: processadaptation_extension_mail_processadaptationmethod_generate_bpmn_tour_J09D_seed_4012


,Meter Reading (MR) Doc. No.,HAUS_session,EQUNR_session,HOUSE_NUM1_session,REGEL,ENTSCHEIDUNG_session,ENTSCHEIDUNG_original


,session_id,process_adaptation,rule_adaptation_method,tour,seed,correct_matches,correct_columns,file_extracted,mails_sent,all_correct
0,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09A,42,True,True,True,True,True
1,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09A,1824,True,True,True,True,True
2,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09A,409,True,True,True,True,True
3,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09A,4506,True,True,True,True,True
4,processadaptation_base_rule_processadaptationm...,base_rule,generate_bpmn,J09A,4012,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
135,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09D,42,True,True,True,True,True
136,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09D,1824,True,True,True,True,True
137,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09D,409,True,True,True,True,True
138,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09D,4506,False,False,False,False,False


,session_id,process_adaptation,rule_adaptation_method,tour,seed,correct_matches,correct_columns,file_extracted,mails_sent,all_correct
58,processadaptation_500_values_processadaptation...,500_values,generate_bpmn,J09D,4506,False,False,False,False,False
75,processadaptation_900_values_processadaptation...,900_values,generate_bpmn,J09D,42,True,False,True,True,False
111,processadaptation_extension_estimates_processa...,extension_estimates,generate_bpmn,J09C,1824,False,False,False,False,False
114,processadaptation_extension_estimates_processa...,extension_estimates,generate_bpmn,J09C,4012,True,True,True,False,False
117,processadaptation_extension_estimates_processa...,extension_estimates,generate_bpmn,J09D,409,False,False,False,False,False
124,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09A,4012,False,False,False,False,False
132,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09C,409,False,False,False,False,False
134,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09C,4012,False,False,False,False,False
138,processadaptation_extension_mail_processadapta...,extension_mail,generate_bpmn,J09D,4506,False,False,False,False,False
